In [30]:
# Importing the Libraries
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input,Embedding,Dense,Flatten,LSTM,GlobalMaxPooling1D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
%%writefile input_data.json
{"intents":
  [
    {
      "tag": "greeting",
      "input": ["hi", "hello", "hey", "howdy"],
      "responses": ["Hello!", "Hi there!", "Hey!", "Howdy!"]
    },
    {
      "tag": "goodbye",
      "input": ["bye", "goodbye", "see you later", "see ya"],
      "responses": ["Goodbye!", "See you later!", "Take care!"]
    },
    {
      "tag": "age",
      "input": ["how old are you?", "what's your age?", "age?"],
      "responses": ["I'm a virtual assistant. I don't have an age."]
    },
    {
      "tag": "weather",
      "input": ["what's the weather like?", "weather forecast?", "how's the weather?"],
      "responses": ["You can check the weather forecast online."]
    },
    {
      "tag": "thanks",
      "input": ["thanks", "thank you", "thanks a lot"],
      "responses": ["You're welcome!", "No problem!", "My pleasure!"]
    },
    {
      "tag": "name",
      "input": ["what's your name?", "name?", "who are you?"],
      "responses": ["I'm a chatbot designed to assist you.", "You can call me ChatBot."]
    },
    {
      "tag": "time",
      "input": ["what time is it?", "current time?", "time?"],
      "responses": ["The current time is [current time]."]
    },
    {
      "tag": "help",
      "input": ["help", "need assistance", "can you help me?"],
      "responses": ["Of course! What do you need help with?", "I'm here to assist you. What can I do for you?"]
    },
    {
      "tag": "favorites",
      "input": ["what's your favorite color?", "favorite movie?", "favorite food?"],
      "responses": ["I don't have personal preferences as I'm a virtual assistant.", "I'm more focused on helping you than having favorites."]
    }
  ]
}


Writing input_data.json


In [3]:
with open('/content/input_data.json') as file:
  data = json.load(file)

In [4]:
tags = []
inputs = []
for values in data.get('intents'):
  tags.append(values.get('tag'))
  inputs.append(values.get('input'))

In [5]:
# Making as DataFrame
df = pd.DataFrame({'inputs':inputs,'tags':tags})

In [6]:
df.head()

,inputs,tags
0,"[hi, hello, hey, howdy]",greeting
1,"[bye, goodbye, see you later, see ya]",goodbye
2,"[how old are you?, what's your age?, age?]",age
3,"[what's the weather like?, weather forecast?, ...",weather
4,"[thanks, thank you, thanks a lot]",thanks


In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
# Remove puncuations
import string
df['inputs'] = df['inputs'].apply(lambda x : ' '.join([value.lower() for value in nltk.word_tokenize(' '.join(x)) if value not in string.punctuation]))

In [9]:
df.head()

,inputs,tags
0,hi hello hey howdy,greeting
1,bye goodbye see you later see ya,goodbye
2,how old are you what 's your age age,age
3,what 's the weather like weather forecast how ...,weather
4,thanks thank you thanks a lot,thanks


In [10]:
# Tokenizing
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(df['inputs'])
train = tokenizer.texts_to_sequences(df['inputs'])
#Sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)


# Label encoder
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
y_train = label.fit_transform(df['tags'])

In [18]:
input_shape = x_train.shape[1]
vocabulary = len(tokenizer.word_index)
output_length = label.classes_.shape[0]

In [28]:
class_label = label.classes_

In [19]:
# Creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation='softmax')(x)
model = Model(i,x)


In [20]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [21]:
model.fit(x_train,y_train,epochs=100)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 2.1953 - accuracy: 0.1111
Epoch 2/100
1/1 [==============================] - 0s 17ms/step - loss: 2.1934 - accuracy: 0.1111
Epoch 3/100
1/1 [==============================] - 0s 15ms/step - loss: 2.1916 - accuracy: 0.3333
Epoch 4/100
1/1 [==============================] - 0s 20ms/step - loss: 2.1897 - accuracy: 0.4444
Epoch 5/100
1/1 [==============================] - 0s 16ms/step - loss: 2.1878 - accuracy: 0.4444
Epoch 6/100
1/1 [==============================] - 0s 16ms/step - loss: 2.1858 - accuracy: 0.4444
Epoch 7/100
1/1 [==============================] - 0s 16ms/step - loss: 2.1839 - accuracy: 0.4444
Epoch 8/100
1/1 [==============================] - 0s 14ms/step - loss: 2.1818 - accuracy: 0.4444
Epoch 9/100
1/1 [==============================] - 0s 17ms/step - loss: 2.1797 - accuracy: 0.4444
Epoch 10/100
1/1 [==============================] - 0s 14ms/step - loss: 2.1776 - accuracy: 0.5556
Epoch 11/100
1/1 [===

In [ ]:
while True:
  text = []
  user = input('You : ')
  if user == 'Sorry':
    print('Please Welcome another day')
    break
  change = lambda x : ' '.join([value.lower() for value in nltk.word_tokenize(' '.join(x)) if value not in string.punctuation])
  text.append(change(user))
  numerical = tokenizer.texts_to_sequences(text)
  data_input = np.array(numerical).reshape(-1)
  padding = pad_sequences([data_input],input_shape)
  predict = model.predict(padding)
  value = np.argmax(predict)
  output_label = class_label[value]
  for choice in data.get('intents'):
    if choice.get('tag') == output_label:
      print(random.choices(choice.get('responses')))

